In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
%pip install -qU transformers evaluate peft seqeval sentencepiece accelerate datasets
import pandas as pd
import regex as re
import numpy as np
from tqdm import tqdm
import string
from collections import defaultdict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s 

In [ ]:
from datasets import load_dataset
# hiner = load_dataset('cfilt/HiNER-collapsed')
hiner = load_dataset('cfilt/HiNER-original')

# label_list = ["B-LOCATION", "B-ORGANIZATION", "B-PERSON", "I-LOCATION", "I-ORGANIZATION","I-PERSON","O"]

label_list = ["B-FESTIVAL","B-GAME","B-LANGUAGE","B-LITERATURE","B-LOCATION","B-MISC","B-NUMEX","B-ORGANIZATION",
              "B-PERSON","B-RELIGION","B-TIMEX","I-FESTIVAL","I-GAME","I-LANGUAGE","I-LITERATURE","I-LOCATION",
              "I-MISC","I-NUMEX","I-ORGANIZATION","I-PERSON","I-RELIGION","I-TIMEX","O"]

id2label = {idx:i for idx, i in enumerate(label_list)}
label2id = {i: idx for idx, i in enumerate(label_list)}

id2label

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{0: 'B-FESTIVAL',
 1: 'B-GAME',
 2: 'B-LANGUAGE',
 3: 'B-LITERATURE',
 4: 'B-LOCATION',
 5: 'B-MISC',
 6: 'B-NUMEX',
 7: 'B-ORGANIZATION',
 8: 'B-PERSON',
 9: 'B-RELIGION',
 10: 'B-TIMEX',
 11: 'I-FESTIVAL',
 12: 'I-GAME',
 13: 'I-LANGUAGE',
 14: 'I-LITERATURE',
 15: 'I-LOCATION',
 16: 'I-MISC',
 17: 'I-NUMEX',
 18: 'I-ORGANIZATION',
 19: 'I-PERSON',
 20: 'I-RELIGION',
 21: 'I-TIMEX',
 22: 'O'}

In [ ]:
vocab = dict()
vocab_count = 0
# vocab["<END>"] = vocab_count
# vocab_count += 1
vocab["<OOV>"] = vocab_count
vocab_count += 1

for sentence in tqdm(iter(hiner['train']["tokens"])):
  for word in sentence:
    if word not in vocab:
      vocab[word] = vocab_count
      vocab_count += 1

75827it [00:00, 185347.72it/s]


In [ ]:
def get_data(data, vocab):
    tokens = []
    tags = []
    for s_token, s_tag in tqdm(zip(data["tokens"], data["ner_tags"])):
        for idx,(i,j) in enumerate(zip(s_token, s_tag)):
            if i in vocab:
                tokens += [i]
                tags += [id2label[j]]
            else:
                tokens += ["<OOV>"]
                tags += [id2label[j]]

    return list(zip(tokens, tags))

In [ ]:
train = get_data(hiner["train"], vocab)
# validation = get_data(hiner["validation"], vocab)
# test = get_data(hiner["test"], vocab)

75827it [00:01, 71729.77it/s]


In [ ]:
import numpy as np

import evaluate
seqeval = evaluate.load("seqeval")

def compute_metrics(labels, predictions):
    # predictions = np.argmax(predictions, axis=2)

    labels = [
        [label_list[l] for l in label]  for label in labels
    ]
    # labels = [
    #     [label_list[l] for (p, l) in zip(prediction, label) if l != (len(label_list)-1)]
    #     for prediction, label in zip(predictions, labels)
    # ]
    # predictions = [
    #     [p             for (p, l) in zip(prediction, label) if l != (len(label_list)-1)]
    #     for prediction, label in zip(predictions, labels)
    # ]

    results = seqeval.compute(predictions=predictions, references=labels, zero_division=1)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
def create_dictionaries(training_corpus, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    prev_tag = 'O'

    i = 0
    for word, tag in tqdm(training_corpus):
        i += 1
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[( tag,word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag

    return emission_counts, transition_counts, tag_counts

In [ ]:
emission_counts, transition_counts, tag_counts = create_dictionaries(train, vocab)

100%|██████████| 1544838/1544838 [00:02<00:00, 547614.99it/s]


In [ ]:
max_index = 0
WORD = None
for i in vocab.keys():
    index = 0
    x = []
    for j in label_list:
        if (j,i) in emission_counts:
            index+=1

    if index>3:
        max_index = index
        x = []
        WORD = i
        for k in label_list:
            if (k,i) in emission_counts:
                x.append(k)

        # print("Word: ", WORD)
        # print("Tags: ", x)
        # print()

In [ ]:
def predict_pos(test, emission_counts, vocab, label_list):
    result = []
    for sentence in tqdm(test):
        if type(sentence)==str:
            sentence = sentence.split()
        predicted = []
        for word in sentence:
            count_final = 0
            pos_final = 'O'
            if word not in vocab:
                word = "<OOV>"
            for pos in label_list:
                key = (pos,word)
                if key in emission_counts:
                    count = emission_counts[key]
                    if count > count_final:
                        count_final = count
                        pos_final = pos
            predicted += [pos_final]
        # print(predicted)
        result.append(predicted)
    return result

In [ ]:
result = predict_pos(hiner["train"]['tokens'], emission_counts, vocab, label_list)
reply = compute_metrics(hiner["train"]['ner_tags'], result)
reply

100%|██████████| 75827/75827 [00:08<00:00, 8637.04it/s]


{'precision': 0.7690518567302042,
 'recall': 0.8254436943284971,
 'f1': 0.7962505826097465,
 'accuracy': 0.9542003757028245}

In [ ]:
# reply['model']="emission"
# reply['split']="test"

# reply = {i:[j] for i,j in reply.items()}

# original = pd.concat([original, pd.DataFrame(reply)], ignore_index=True)
# original

In [ ]:
# original = pd.DataFrame()

In [ ]:
def create_transition_matrix(alpha, label_list, tag_counts, transition_counts):
    num_tags = len(label_list)
    A = np.zeros((num_tags,num_tags))
    trans_keys = set(transition_counts.keys())
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (label_list[i],label_list[j])
            if key in transition_counts:
                count = transition_counts[key]
            count_prev_tag = tag_counts[label_list[i]]
            A[i,j] = (count + alpha) / (count_prev_tag + alpha*num_tags)
    return A

def create_emission_matrix(alpha, label_list, tag_counts, emission_counts, vocab):
    num_tags = len(tag_counts)
    num_words = len(vocab)
    B = np.zeros((num_tags, num_words))
    emis_keys = set(list(emission_counts.keys()))
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key = (label_list[i],vocab[j])
            if key in emission_counts.keys():
                count = emission_counts[key]
            count_tag = tag_counts[label_list[i]]
            B[i,j] = (count + alpha) / (count_tag+ alpha*num_words)
    return B

In [ ]:
A = create_transition_matrix(1e-15, label_list, tag_counts, transition_counts)
B = create_emission_matrix(1e-15, label_list, tag_counts, emission_counts, list(vocab))

# Beam Search

In [ ]:
def beam_search(test, beam_width, A, B, label_list, vocab):
    result = []
    for sentence in tqdm(test):
        if type(sentence)==str:
            sentence = sentence.split()

        sentence = [i if i in vocab else "<OOV>" for i in sentence]

        beam = [([], 0)]
        for word in sentence:
            new_beam = []
            for sequence, score in beam:
                prev_label_index = None
                for label_index, label in enumerate(label_list):
                    if len(sequence)==0:
                        prev_label_index = len(label_list)-1
                    else:
                        prev_label_index = label_list.index(sequence[-1])
                    emission_score = B[label_index][vocab[word]]
                    transition_score = A[prev_label_index][label_index]
                    new_score = score + np.log(emission_score) + np.log(transition_score)

                    new_sequence = sequence + [label,]
                    new_beam.append((new_sequence, new_score))

            new_beam.sort(key=lambda x: x[1], reverse=True)
            beam = new_beam[:beam_width]
        result.append(beam)
    return result

In [ ]:
# result = beam_search(hiner["test"]['tokens'][92:93],5, A,B,label_list,vocab)
# reply = compute_metrics(hiner["test"]['ner_tags'], result)
# reply
print()
print("Tokens: ",hiner["test"]['tokens'][92:93][0])
print()
print("Beam Search results:")
for i in result[0]:
    print(i)



Tokens:  ["'", 'बीबीसी', 'चुनाव', 'एक्सप्रेस', "'", 'आज', 'से', '.']

Beam Search results:
(['O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O', 'O', 'O', 'O'], -54.404878561023004)
(['O', 'B-ORGANIZATION', 'O', 'O', 'O', 'O', 'O', 'O'], -54.664195253624946)
(['O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'O', 'O', 'O', 'O', 'O'], -56.470701138190286)
(['O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O', 'B-TIMEX', 'O', 'O'], -56.8058407469366)
(['O', 'B-ORGANIZATION', 'O', 'O', 'O', 'B-TIMEX', 'O', 'O'], -57.065157439538545)


# Virterbi

In [ ]:
def initialize(A, B, input, label_list, tag_counts, vocab):
    num_tags = len(tag_counts)
    best_probs = np.zeros((num_tags, len(input)))
    best_paths = np.zeros((num_tags, len(input)), dtype=int)
    s_idx = label_list.index("O")
    for i in range(num_tags):
        if A[s_idx,i] == 0:
            best_probs[i,0] = float('-inf')
        else:
            best_probs[i,0] = np.log(A[s_idx,i]) + np.log(B[i,vocab[input[0]]])
    return best_probs, best_paths

def viterbi_forward(A, B, input, best_probs, best_paths, vocab):
    num_tags = best_probs.shape[0]
    for i in range(1, len(input)):
        for j in range(num_tags):
            best_prob_i =  float("-inf")
            best_path_i = None
            for k in range(num_tags):
                prob = best_probs[k,i-1]+np.log(A[k,j]) +np.log(B[j,vocab[input[i]]])
                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
            best_probs[j,i] = best_prob_i
            best_paths[j,i] = best_path_i
    return best_probs, best_paths

def viterbi_backward(best_probs, best_paths, label_list):
    m = best_paths.shape[1]
    num_tags = best_probs.shape[0]
    best_prob_for_last_word = float('-inf')
    pred = [None] * m
    pos_tag_for_word_i = np.argmax(best_probs[:,-1])
    pred[m - 1] = label_list[pos_tag_for_word_i]

    for i in range(m-1, 0, -1):
        pos_tag_for_word_i = best_paths[pos_tag_for_word_i,i]
        pred[i - 1] = label_list[pos_tag_for_word_i]
    return pred

In [ ]:
def viterbi(data, A, B, label_list, tag_counts, vocab):
    outputs = []
    s = 0
    inputs = []
    for sentence in tqdm(data):
        if type(sentence)==str:
            sentence = sentence.split()
        sentence = [i if i in vocab else "<OOV>" for i in sentence]
        inputs.append(sentence)
        best_probs, best_paths = initialize(A, B, sentence, label_list, tag_counts, vocab)
        best_probs, best_paths = viterbi_forward(A, B, sentence, best_probs, best_paths, vocab)
        output = viterbi_backward(best_probs, best_paths, label_list)

        outputs.append(output)
    return inputs, outputs

In [ ]:
ans =  viterbi([" ".join(hiner["test"]["tokens"][9])], A, B, label_list, tag_counts, vocab)
reply = compute_metrics(hiner["test"]["ner_tags"],ans[1])
# ans

100%|██████████| 1/1 [00:00<00:00, 109.45it/s]

[('चेन्नई', 'B-LOCATION'),
 ('में', 'O'),
 ('14', 'O'),
 ('अक्तूबर', 'O'),
 ('को', 'O'),
 ('श्रीलंका', 'B-LOCATION'),
 ('के', 'O'),
 ('तमिलों', 'O'),
 ('के', 'O'),
 ('मुद्दे', 'O'),
 ('पर', 'O'),
 ('सर्वदलीय', 'O'),
 ('बैठक', 'O'),
 ('हुई', 'O'),
 ('थी', 'O'),
 ('.जिसमें', 'O'),
 ('यह', 'O'),
 ('फ़ैसला', 'O'),
 ('हुआ', 'O'),
 ('था', 'O'),
 ('कि', 'O'),
 ('अगर', 'O'),
 ('29', 'O'),
 ('अक्तूबर', 'O'),
 ('तक', 'O'),
 ('केंद्र', 'B-ORGANIZATION'),
 ('सरका', 'I-ORGANIZATION'),
 ('र', 'O'),
 ('श्रीलंका', 'B-LOCATION'),
 ('के', 'O'),
 ('उत्तर', 'B-LOCATION'),
 ('में', 'O'),
 ('संघर्षविराम', 'O'),
 ('करवाने', 'O'),
 ('के', 'O'),
 ('लिए', 'O'),
 ('क़दम', 'O'),
 ('नहीं', 'O'),
 ('उठाती', 'O'),
 ('तो', 'O'),
 ('राज्य', 'O'),
 ('के', 'O'),
 ('सभी', 'O'),
 ('सांसद', 'O'),
 ('इस्तीफ़ा', 'O'),
 ('दे', 'O'),
 ('देंगे', 'O'),
 ('.', 'O')]

In [ ]:
reply['model']="viterbi"
reply['split']="test"

reply = {i:[j] for i,j in reply.items()}

original = pd.concat([original, pd.DataFrame(reply)], ignore_index=True)
original

,precision,recall,f1,accuracy,model,split
0,0.824051,0.870495,0.846636,0.970821,emission,train
1,0.789533,0.744853,0.766542,0.950415,emission,validation
2,0.783849,0.745795,0.764349,0.950704,emission,test
3,0.919479,0.915018,0.917243,0.982069,viterbi,train
4,0.860013,0.83922,0.84949,0.962522,viterbi,validation
5,0.855622,0.84115,0.848324,0.962123,viterbi,test


In [ ]:
original.to_excel("collapseddd.xlsx", index=None)